<a href="https://colab.research.google.com/github/MateuszOszczypala/Data_public/blob/main/Gravitational_search_algorithm_RAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
from google.colab import drive
from itertools import combinations
from scipy.spatial.distance import pdist

In [ ]:
# Basic parameters of GSA
number_of_iterations = 10000
swarm_size = 200
lb = 0
ub = 255
lb2 = -0.5
ub2 = 5.5-10e-06
alpha = 0.10
G0 = 1000

In [ ]:
# Basic parameters of system
number_of_subsystems = 10
working_failure_rate = [0.8147, 0.9058, 0.1270, 0.9134, 0.6324, 0.0975, 0.2785, 0.5469, 0.9575, 0.9649]
standby_failure_rate = [0.0943, 0.1049, 0.0147, 0.1057, 0.0732, 0.0113, 0.0322, 0.0633, 0.1108, 0.1117]
repair_rate = [2.9412, 2.9143, 1.9708, 2.6006, 1.2838, 1.8435, 2.8315, 2.5844, 2.9190, 2.3115]
warm_standby_switching_rate = [5.3571, 13.4913, 14.3399, 11.7874, 2.5774, 12.4313, 8.9223, 11.5548, 6.7119, 12.0605]
cold_standby_switching_rate = [2.6785, 6.7457, 7.1700, 5.8937, 1.2887, 6.2157, 4.4611, 5.7774, 3.3559, 6.0302]
k = [3, 3, 1, 1, 2, 2, 1, 1, 3, 3]
unit_cost = [5.67, 8.36, 8.57, 4.31, 6.43, 6.12, 7.52, 7.97, 8.28, 4.93]
unit_weight = [5.22, 6.94, 5.32, 5.68, 10.76, 9.86, 7.22, 11.65, 5.24, 8.07]

In [ ]:
# Constraints
weight_constraint = 400
cost_constraint = 350

In [ ]:
def Create_CTMC_model_no_redundancy(n, working_failure_rate, repair_rate): # strategy 0
  # Initialize CTMC matrix
  CTMC = np.zeros((2, 2))
  # State no. 0 is subsystem working state
  # State no. 1 is subsystem failure state

  # Transition rates
  CTMC[0, 1] = n*working_failure_rate # Failure of working component
  CTMC[1, 0] = repair_rate # Repair
  CTMC[0, 0] = -n*working_failure_rate # diagonal
  CTMC[1, 1] = -repair_rate # diagonal
  return CTMC

In [ ]:
def Create_CTMC_model_hot_standby(n, k, working_failure_rate, repair_rate): # strategy 1
  State_space = np.empty((n-k+2, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # State no. n-k+1 is subsystem failure state

  # Working states with k working components and remaining in hot standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
    State_space[n-k+1, 0] = k-1 # number of working components
    State_space[n-k+1, 1] = 0 # number of hot standby components
    State_space[n-k+1, 2] = n-k+1 # number of failed components

  # Initialize CTMC matrix
  CTMC = np.zeros((n-k+2, n-k+2))

  # Transition rates
  for i in range(n-k+2):
    for j in range(n-k+2):
      if State_space[i, 0] - State_space[j, 0] == 1: # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 2] - State_space[j, 2] == 1: # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(n-k+2):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_warm_standby(n, k, working_failure_rate, standby_failure_rate, repair_rate, switching_rate): # strategy 2
  State_space = np.empty((2*n-2*k+3, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # States no. form n-k+1 to 2n-2k+2 are subsystem failure states

  # Working states with k working components and remaining in warm standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of warm standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
  j = 0
  for i in range(n-k+1, 2*n-2*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of warm standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((2*n-2*k+3, 2*n-2*k+3))

  # Transition rates
  for i in range(2*n-2*k+3):
    for j in range(2*n-2*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of warm standby component
        if State_space[i, 0] == k - 1:
          # CTMC[i, j] = (State_space[i, 1] + State_space[i, 0]) * standby_failure_rate
          CTMC[i, j] = 0 # according to the assumption of non-failing components in subsystem failure state
        else:
          CTMC[i, j] = State_space[i, 1] * standby_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(2*n-2*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_cold_standby(n, k, working_failure_rate, repair_rate, switching_rate): # strategy 3
  State_space = np.empty((2*n-2*k+3, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # States no. form n-k+1 to 2n-2k+2 are subsystem failure states

  # Working states with k working components and remaining in cold standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of cold standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
  j = 0
  for i in range(n-k+1, 2*n-2*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of cold standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((2*n-2*k+3, 2*n-2*k+3))

  # Transition rates
  for i in range(2*n-2*k+3):
    for j in range(2*n-2*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(2*n-2*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_mixed_strategy_active_warm(n, k, working_failure_rate, standby_failure_rate, repair_rate, switching_rate): # strategy 4
  State_space = np.empty((3*n-3*k+3, 3), dtype=int)

  j = 0
  # Working states with k+1 active components
  for i in range(n-k):
    State_space[i, 0] = k+1 # number of working components
    State_space[i, 1] = n-k-1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  j = 0
  # Working states with k active components
  for i in range(n-k, 2*n-2*k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  j = 0
    # Failed states with k-1 active components
  for i in range(2*n-2*k+1, 3*n-3*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((3*n-3*k+3, 3*n-3*k+3))

  # Transition rates
  for i in range(3*n-3*k+3):
    for j in range(3*n-3*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of active component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of warm standby component
        if State_space[i, 0] == k - 1:
          # CTMC[i, j] = (State_space[i, 1] + State_space[i, 0]) * standby_failure_rate
          CTMC[i, j] = 0 # according to the assumption of non-failing components in subsystem failure state
        else:
          CTMC[i, j] = State_space[i, 1] * standby_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = (min(min(State_space[i, 1], 2), k + 1 - State_space[i, 0])) * switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(3*n-3*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_mixed_strategy_active_cold(n, k, working_failure_rate, repair_rate, switching_rate): # strategy 5
  State_space = np.empty((3*n-3*k+3, 3), dtype=int)

  j = 0
  # Working states with k+1 active components
  for i in range(n-k):
    State_space[i, 0] = k+1 # number of working components
    State_space[i, 1] = n-k-1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  j = 0
  # Working states with k active components
  for i in range(n-k, 2*n-2*k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  j = 0
    # Failed states with k-1 active components
  for i in range(2*n-2*k+1, 3*n-3*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((3*n-3*k+3, 3*n-3*k+3))

  # Transition rates
  for i in range(3*n-3*k+3):
    for j in range(3*n-3*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of active component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = (min(min(State_space[i, 1], 2), k + 1 - State_space[i, 0])) * switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(3*n-3*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Ergodic_probabilities(CTMC):
  CTMC_transposed = CTMC.T
  CTMC_transposed[0, :CTMC_transposed.shape[0]] = 1
  P = np.zeros((CTMC_transposed.shape[0], 1))
  P[0, 0] = 1
  Ergodic_prob = np.linalg.inv(CTMC_transposed) @ P
  return Ergodic_prob

In [ ]:
def Availability_of_subsystems(n, k, Ergodic_prob, redundancy_strategy):

  if redundancy_strategy == 0: # no redundancy
    Subsystem_availabilities = np.sum(Ergodic_prob[0:1])

  elif redundancy_strategy == 1: # hot standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 2: # warm standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 3: # cold standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 4: # mixed strategy(active + warm)
    Subsystem_availabilities = np.sum(Ergodic_prob[0:2*n-2*k+1])

  elif redundancy_strategy == 5: # mixed strategy(active + cold)
    Subsystem_availabilities = np.sum(Ergodic_prob[0:2*n-2*k+1])

  return Subsystem_availabilities

In [ ]:
def objective_function(Subsystem_availabilities):
  A = Subsystem_availabilities
  U = 1 - A
  System_availability = (
        A[0]*A[1]*A[2]*A[3]+
        A[0]*A[1]*A[5]*A[9]*(U[2]+A[2]*U[3])+
        A[0]*A[4]*A[8]*A[9]*(U[1]+A[1]*U[2]*U[5]+A[1]*A[2]*U[3]*U[5])+
        A[6]*A[7]*A[8]*A[9]*(U[0]+A[0]*U[1]*U[4]+A[0]*A[1]*U[2]*U[4]*U[5]+A[0]*A[1]*A[2]*U[3]*U[4]*U[5])+
        A[1]*A[2]*A[3]*A[4]*A[6]*A[7]*U[0]*(U[8]+A[8]*U[9])+
        U[0]*A[2]*A[3]*A[5]*A[6]*A[7]*A[8]*U[9]*(U[1]+A[1]*U[4])+
        A[0]*U[1]*A[2]*A[3]*A[5]*A[6]*A[7]*A[8]*A[9]*U[9]+
        A[0]*U[1]*A[2]*A[3]*A[4]*A[5]*A[8]*U[9]*(U[6]+A[6]*U[7])+
        U[0]*A[1]*A[4]*A[5]*A[6]*A[7]*A[8]*U[8]*A[9]*(U[2]+A[2]*U[3])
    )
  return System_availability

In [ ]:
def component_allocation(redundancy_strategy, priority, weight_available, unit_weight, cost_available, unit_cost):
  a = np.zeros(priority.shape[0])
  for i in range(priority.shape[0]):
    a[i] = priority[i]/255
  standby_components = np.zeros(priority.shape[0], dtype=int)

  for i in range(priority.shape[0]):
    if redundancy_strategy[i] == 0: # no redundancy
      standby_components[i] = 0
    else:
      standby_components[i] = round(a[i] * min(round(weight_available/unit_weight[i]),round(cost_available/unit_cost[i])))
  return standby_components

In [ ]:
# Initialization
def initialization(number_of_subsystems, swarm_size, lb, ub, lb2, ub2):
  initial_swarm = np.empty((4, number_of_subsystems, swarm_size), dtype=float)
  initial_swarm[0, :, :] = np.random.uniform(lb, ub, (1, number_of_subsystems, swarm_size))
  initial_swarm[1, :, :] = np.random.uniform(lb2, ub2, (1, number_of_subsystems, swarm_size))
  initial_swarm[2, :, :] = np.random.uniform(lb, ub, (1, number_of_subsystems, swarm_size))
  initial_swarm[3, :, :] = np.random.uniform(lb2, ub2, (1, number_of_subsystems, swarm_size))

  return initial_swarm

In [ ]:
# Gravitational search algorithm
agent_positions = np.empty((2, number_of_subsystems, swarm_size, number_of_iterations), dtype=float)
agent_velocities = np.empty((2, number_of_subsystems, swarm_size, number_of_iterations), dtype=float)
objective_function_value = np.full((swarm_size, number_of_iterations), np.nan)
n = np.full((number_of_subsystems, swarm_size, number_of_iterations), np.nan)
redundancy_strategy = np.full((number_of_subsystems, swarm_size, number_of_iterations), np.nan)
best_availability = np.full((number_of_iterations), np.nan)
best_position = np.full((2, number_of_subsystems, number_of_iterations), np.nan)
worst_availability = np.full((number_of_iterations), np.nan)
G_allocation = np.full((number_of_iterations), np.nan)
G_strategy = np.full((number_of_iterations), np.nan)
agent_masses = np.full((swarm_size, number_of_iterations), np.nan)
agent_normalized_masses = np.full((swarm_size, number_of_iterations), np.nan)
gravitational_forces = np.zeros((2, number_of_subsystems, swarm_size, number_of_iterations), dtype=float)

# Initialization
initial_swarm = initialization(number_of_subsystems, swarm_size, lb, ub, lb2, ub2)
agent_positions[:,:,:,0] = initial_swarm[0:2,:,:]
agent_velocities[:,:,:,0] = initial_swarm[2:4,:,:]

available_weight = 0
for i in range(number_of_subsystems):
  available_weight += k[i]*unit_weight[i]

available_cost = 0
for i in range(number_of_subsystems):
  available_cost += k[i]*unit_cost[i]

for num_it in range(number_of_iterations):
  for num_agent in range(swarm_size):

        # identify the redundancy strategy
    for i in range(number_of_subsystems):
      redundancy_strategy[i, num_agent, num_it] = round(agent_positions[1, i, num_agent, num_it])

       # allocate standby components to subsystems
    standby_components = component_allocation(redundancy_strategy[:, num_agent, num_it], agent_positions[0, :, num_agent, num_it], available_weight, unit_weight, available_cost, unit_cost)
    n_components = np.zeros(number_of_subsystems, dtype=int)
    for i in range(number_of_subsystems):
      n[i, num_agent, num_it] = k[i] + int(standby_components[i])
      n_components[i] = k[i] + int(standby_components[i])

    # calculate subsystem availabilities
    Subsystem_availabilities = np.full((number_of_subsystems), np.nan)

    for num_subsystem in range(number_of_subsystems):
      if redundancy_strategy[num_subsystem, num_agent, num_it] == 0:
        CTMC = Create_CTMC_model_no_redundancy(n_components[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_agent, num_it] == 1:
        CTMC = Create_CTMC_model_hot_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_agent, num_it] == 2:
        CTMC = Create_CTMC_model_warm_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], standby_failure_rate[num_subsystem], repair_rate[num_subsystem], warm_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_agent, num_it] == 3:
        CTMC = Create_CTMC_model_cold_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem], cold_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_agent, num_it] == 4:
        CTMC = Create_CTMC_model_mixed_strategy_active_warm(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], standby_failure_rate[num_subsystem], repair_rate[num_subsystem], warm_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_agent, num_it] == 5:
        CTMC = Create_CTMC_model_mixed_strategy_active_cold(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem], cold_standby_switching_rate[num_subsystem])

      Ergodic_prob = Ergodic_probabilities(CTMC)
      Subsystem_availabilities[num_subsystem] = Availability_of_subsystems(n_components[num_subsystem], k[num_subsystem], Ergodic_prob, redundancy_strategy[num_subsystem, num_agent, num_it])

      del CTMC, Ergodic_prob
    # objective value (system availability) calculation
    if np.sum(n[:, num_agent, num_it]*unit_cost) > cost_constraint or np.sum(n[:, num_agent, num_it]*unit_weight) > weight_constraint:
      num_components = np.full((number_of_subsystems), np.nan)
      for i in range(number_of_subsystems):
        num_components[i] = n[i, num_agent, num_it]
        system_cost = 0
      for i in range(number_of_subsystems):
        system_cost = system_cost + num_components[i]*unit_cost[i]
        system_weight = 0
      for i in range(number_of_subsystems):
        system_weight = system_weight + num_components[i]*unit_weight[i]
      if system_cost/cost_constraint > system_weight/weight_constraint:
        penalty_function = 0.01**(system_cost/cost_constraint)
      else:
        penalty_function = 0.01**(system_weight/weight_constraint)
      objective_function_value[num_agent, num_it] = objective_function(Subsystem_availabilities)*penalty_function
    else:
      objective_function_value[num_agent, num_it] = objective_function(Subsystem_availabilities)

  best_availability[num_it] = np.max(objective_function_value[:, :num_it+1])
  worst_availability[num_it] = np.min(objective_function_value[:, :num_it+1])

  best_agent, best_index = np.unravel_index(np.argmax(objective_function_value[:, :num_it+1]), objective_function_value[:, :num_it+1].shape)
  best_position[:, :, num_it] = agent_positions[:, :, best_agent, best_index]

  # Update the gravitational constant for component allocation
  G_allocation[num_it] = G0*((1/(num_it+1))**alpha)
  # Update the gravitational constant for redundancy strategies
  G_strategy[num_it] = (ub2/ub)*G0*((1/(num_it+1))**alpha)

  # Calculate masses of agents
  for i in range(swarm_size):
    if best_availability[num_it] > worst_availability[num_it]:
      agent_masses[i, num_it] = (objective_function_value[i, num_it]-worst_availability[num_it])/(best_availability[num_it]-worst_availability[num_it]) + 1e-08
    else:
      agent_masses[i, num_it] = 1

  # Normalize masses
  for i in range(swarm_size):
    agent_normalized_masses[i, num_it] = agent_masses[i, num_it]/np.sum(agent_masses[:, num_it])

  # Update gravitational forces
  for i in range(swarm_size):
    for j in range(swarm_size):
      if i != j:
        agent_distance_allocation = math.sqrt(sum((agent_positions[0, :, i, num_it] - agent_positions[0, :, j, num_it])**2)) + 1e-8  # Distance between agents
        agent_distance_strategy = math.sqrt(sum((agent_positions[1, :, i, num_it] - agent_positions[1, :, j, num_it])**2)) + 1e-8 # Distance between agents
        force_allocation = G_allocation[num_it] * (agent_normalized_masses[i, num_it] * agent_normalized_masses[j, num_it]) / agent_distance_allocation # Gravitational force magnitude
        force_strategy =  G_strategy[num_it] * (agent_normalized_masses[i, num_it] * agent_normalized_masses[j, num_it]) / agent_distance_strategy # Gravitational force magnitude
        direction_allocation = (agent_positions[0, :, j, num_it] - agent_positions[0, :, i, num_it]) # Direction of force
        direction_strategy = (agent_positions[1, :, j, num_it] - agent_positions[1, :, i, num_it]) # Direction of force
        gravitational_forces[0, :, i, num_it] = gravitational_forces[0, :, i, num_it] + force_allocation * direction_allocation     # Update total force
        gravitational_forces[1, :, i, num_it] = gravitational_forces[1, :, i, num_it] + force_strategy * direction_strategy     # Update total force

  # Update velocities and positions
  for i in range(swarm_size):
    acceleration = gravitational_forces[:, :, i, num_it] / (agent_normalized_masses[i, num_it] + 1e-08)
    if num_it < number_of_iterations - 1:
      if objective_function_value[i, num_it] < best_availability[num_it]:
        agent_velocities[:, :, i, num_it+1] = np.random.uniform(0, 1)*agent_velocities[:, :, i, num_it] + acceleration
        agent_positions[:, :, i, num_it+1] = agent_positions[:, :, i, num_it] + agent_velocities[:, :, i, num_it+1]
      else:
        agent_velocities[:, :, i, num_it+1] = 0
        agent_positions[:, :, i, num_it+1] = agent_positions[:, :, i, num_it]

  # Apply the boundaries
  if num_it < number_of_iterations - 1:
    for i in range(swarm_size):
      agent_positions[0, :, i, num_it+1] = np.clip(agent_positions[0, :, i, num_it+1], lb, ub)
      agent_positions[1, :, i, num_it+1] = np.clip(agent_positions[1, :, i, num_it+1], lb2, ub2)

In [ ]:
plt.plot(np.max(objective_function_value, axis=0))
plt.xlabel('Number of generation')
plt.ylabel('System availability')
plt.show()

In [ ]:
print(agent_positions[:,:,0,99])

In [ ]:
print(n[:,0,99])

In [ ]:
print(redundancy_strategy[:,0,99])

In [ ]:
objective_function_value[0,0]

In [ ]:
agent_positions[:,:,0,0]

In [ ]:
print(agent_distance_allocation)
print(agent_distance_strategy)

In [ ]:
print(gravitational_forces[:,:,0,2])

In [ ]:
print(force_allocation)
print(force_strategy)

In [ ]:
print(acceleration)

In [ ]:
print(agent_normalized_masses[0,0])

In [ ]:
print(gravitational_forces[:,:,0,0])

In [ ]:
plt.plot(G_allocation)
plt.plot(G_strategy)
plt.xlabel('Number of generation')
plt.ylabel('System availability')
plt.show()